# YiVal Prompt Retrieval ⚙️ ⚙️

### **What is YiVal?**
> YiVal is a versatile platform support customize test data, evaluation methods and enhancement strategy , all in one.
It enpowers you to generate better results, reduce latency and decrease inference cost.

**~~TL~~DR**: YiVal streamlines the **evaluation** and **enhancement** of GenAI Apps, enhance ane evaluate **everything** with ease.

### **Why YiVal**


*   Native support **Multi-modal** apps: text📄 + audio🎙 + image🌃 + video🎥
*   **Multi-components**: which doesn't even have to be GenAI 😁
*   Native **RLHF** and **RLAIF** ⚙️
*   Most advanced open source **enhancement algorithms** 🪄


# **Install the latest yival with git**

We provide two ways of yival install

1. install with pip
```
pip install yival
```

2. Developer Mode: The latest yival
```
git clone https://github.com/YiVal/YiVal.git
poetry config virtualenvs.create true
poetry install
```

here we install with poetry , you can find the detail below
* install poetry in colab environment
* install yival with poetry

In [ ]:
# clone the latest yival
import os
!python --version
!rm -rf YiVal
!git clone -b -b stable https://github.com/YiVal/YiVal.git

# install and config poetry
import shutil
!pip install poetry
POETRY_PATH = shutil.which("poetry") or (os.getenv("HOME") + "/.local/bin/poetry")
os.environ["PATH"] += os.pathsep + os.path.dirname(POETRY_PATH)
!poetry --version
!poetry config virtualenvs.create true

In [ ]:
os.chdir("/content/YiVal")
!poetry install --no-ansi

# **Configure your OpenAI API key**

In yival, most of time we will use chatgpt as data_generator, evaluator , improver...

It's really amazing to find that llm is so powerful

In some of the demos provided by Yival, we use GPT-4 for data generation, result evaluation, and so on, because GPT-4 has stronger **reasoning** capabilities.


In [ ]:
os.environ['OPENAI_API_KEY'] = ''

# **[Optional] Change gpt-4 to gpt-3.5-turbo in config**

If you don't have a GPT-4 account, you can also use GPT-3.5-turbo to complete the entire process, you just need to modify the **model_name** in the config file.

For example , you can find `model_name` below

```yaml
description: Generate test data
dataset:
  data_generators:
    openai_prompt_data_generator:
      chunk_size: 100000
      diversify: true
      model_name: gpt-4 #Change the model_name to gpt-3.5-turbo here 🦄️
      input_function:
        description:
          Given a tech startup business, generate a corresponding landing
          page headline
        name: headline_generation_for_business
        parameters:
          tech_startup_business: str
      number_of_examples: 3
      output_csv_path: generated_examples.csv
  source_type: machine_generated
```

If you want to use gpt-3.5-turbo, change the `use_gpt_35_turbo` to `True` in the below cell and run it.

It will autotimatically replace all `gpt-4` to `gpt-3.5-turbo` in all yamls provided by yival

In [ ]:
import os, glob, yaml
use_gpt_35_turbo = True  #change it to True if you don't want to use gpt-4

def replace_gpt4_recursive(data):
    if isinstance(data, str):
        return data.replace('gpt-4', 'gpt-3.5-turbo')
    elif isinstance(data, list):
        return [replace_gpt4_recursive(item) for item in data]
    elif isinstance(data, dict):
        return {key: replace_gpt4_recursive(value) for key, value in data.items()}
    else:
        return data


def replace_in_yaml_files(directory):
    for filename in glob.glob(os.path.join(directory, '*.yml')):
        with open(filename, 'r') as file:
            data = yaml.safe_load(file)
        data = replace_gpt4_recursive(data)
        with open(filename, 'w') as file:
            yaml.safe_dump(data, file)

if use_gpt_35_turbo:
  replace_in_yaml_files("/content/YiVal/demo/configs")
  print("[INFO] replace all gpt-4 to gpt-3.5-turbo. Use gpt-3.5-turbo in the coming page")
else:
  print("[INFO] use default gpt-4")


# Simplifying Prompt Generation for LLMs with Retrieval Methods

Working with Large Language Models (LLMs) like ChatGPT often involves feeding
them prompts: short texts that give the model direction on how to respond.
Manually crafting these prompts can be tedious. Wouldn't it be convenient if we
could automate this process, ensuring each prompt is contextually relevant?
Let's explore a method that does just that, using the Yival framework and the
FAISS vector database.

## **Why Automate Prompt Generation?**

Think of prompts as questions or instructions you give to ChatGPT. The more
precise the instruction, the better the model's answer. But crafting a new
instruction for every unique scenario is time-consuming. If we could automate
this, not only would it save time, but it'd also ensure the AI's responses are
consistently relevant.

## **Storing Prompts: FAISS to the Rescue**

We source a wide variety of prompts from [awesome-chatgpt-prompts](https://github.com/f/awesome-chatgpt-prompts).
To quickly find the best prompt for any situation:

- **We use FAISS**: This tool helps store prompts in a way that makes them quick
  to search and retrieve.
- **We turn prompts into 'vectors'**: By transforming prompts into a mathematical
  format (vectors), we can easily find the most fitting one for a given situation.

## **How Yival Helps in Retrieval**

Yival is like a toolbox that simplifies AI-related experimentation.
When integrated with FAISS, it streamlines the process of fetching the right prompt.

### **The Main Steps**

1. **Find Matching Prompts**: Based on a given situation, Yival searches the FAISS
    database to find similar prompts.
2. **Refine with GPT**: Sometimes, the initially found prompts might not be perfect.
    So, we use GPT to rerank them, ensuring we pick the most suitable one.

## **Putting It All Together**

With our setup, generating a prompt becomes straightforward. For ChatGPT,
here's a simple example of how we might use a generated prompt:

```python
# Create a chat message sequence
messages = [{
    "role": "user",
    "content": str(StringWrapper("", name="prompt")) + f'\n{input}'
}]
# Get a response from ChatGPT
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo", messages=messages
)
```

Notice that we didn't need to provide a predefined prompt. The system took care
of it!

## **A Peek at the Process**

### **Visual Flow**

<img src="https://uninaruto.oss-cn-shanghai.aliyuncs.com/img/20230904154944.png" width="80%" >


This flowchart will give you a bird's-eye view of how everything connects, from the moment we receive a use-case to generating the perfect prompt.

### **Prompts Retrivel Yaml**

```
custom_function: demo.auto_prompt_bot.reply
dataset:
  source_type: user_input
description: Basic config for interactive mode
evaluators: []

custom_variation_generators:
  retrivel_variation_generator:
    class: demo.prompts_retrivel.retrivel_variation_generator.RetrivelVariationGenerator
    config_cls: demo.prompts_retrivel.retrivel_variation_generator_config.RetrivelVariationGeneratorConfig

variations:
  - name: prompt
    generator_config:
      use_case: "travel guide"
    generator_name: retrivel_variation_generator
```
#### Dataset
The experiment sources its data from user input, which is from interactive mode in dash app.

#### Custom Function
The custom function `demo.auto_prompt_bot.reply` is utilized in this experiment. This function takes a string 'input' as parameter, and generates a response using OpenAI's gpt-4 model.

#### Variations
In this experiment, we uses `retrivel_variation_generator` to generate a prompt related to `travel guide` use case.
### **Results in Real-Time**

![Output Screenshot](https://github.com/YiVal/YiVal/assets/1544154/35216786-ac3c-4884-8818-68647511228d)

Here's an example of what the system's output looks like in action. This gives
you an idea of the kind of prompts it can generate and how ChatGPT might respond.

## **In Conclusion**

The world of AI is vast and sometimes complex. But tools like Yival and FAISS,
when combined with LLMs like ChatGPT, can make tasks like prompt generation much
simpler. By automating this process, we're taking a step towards more efficient
and context-aware AI interactions.

You can review the full code [here](https://github.com/YiVal/YiVal/tree/master/demo/prompts_retrivel)


# Configure your Ngrok token
Our current ngrok authtoken only supports one public session at a time. If it's being used by others or if you're using it to run multiple Colabs at once, you might bump into a Network error. To avoid this, we suggest getting your own ngrok authtoken for your Colab notebooks. It's easy and free to get your own authtoken from ngrok.

Here's how to do it:
- If you don't have a ngrok account yet, head over to https://dashboard.ngrok.com/login to sign up.
- Once you're logged in, you can grab your authtoken at https://dashboard.ngrok.com/get-started/your-authtoken.

Prior to initiating a new demo, ensure that all other applications utilizing ngrok within Colab have been terminated via the `Connect -> Manage Sessions` pathway. You can check and manage your sessions as follow picture.

<img src="https://github.com/uni-zhuan/uni_CDN/blob/master/picture/Yival/iShot_2023-10-12_22.51.49.png?raw=true" width="80%" height="50%">

In [ ]:
os.environ['ngrok']='true'
!poetry run ngrok config add-authtoken 2UK3G7MKgDqCqDnu36njaaE02bZ_7FqvcqBke5hbpgHjizoo7

# YIVAL！

Test from the interactive mode page on the web, input the parameters you want (such as San Francisco in our example) and select combinations. Click run, then wait to check the corresponding results.

In [ ]:
!poetry run yival run /content/YiVal/demo/configs/config_prompts_retrivel.yml